In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
EP6 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP6_total2.csv")
EP7 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP7_total2.csv")
EP8 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP8_total2.csv")


In [ ]:
import pandas as pd
EP6_2 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP6_RCVs.csv")
EP7_2 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP7_RCVs.csv")
EP8_2 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP8_RCVs.csv")

In [ ]:
EP6_2.head()

,Unnamed: 0,WebisteEpID,Fname,Lname,FullName,Activ,Country,Party,EPG,Start,...,6191,6192,6193,6194,6195,6196,6197,6198,6199,6200
0,0,28469,Adamos,ADAMOU,"ADAMOU, Adamos",yes,Cyprus,Anorthotiko Komma Ergazomenou Laou - Aristera ...,Confederal Group of the European United Left -...,2004-07-20,...,5,5,5,5,5,5,5,5,5,5
1,1,28302,Filip,ADWENT,"ADWENT, Filip",no,Poland,Liga Polskich Rodzin,Independence/Democracy Group,2004-07-20,...,0,0,0,0,0,0,0,0,0,0
2,2,28975,Vittorio,AGNOLETTO,"AGNOLETTO, Vittorio",yes,Italy,Partito della Rifondazione Comunista - Sinistr...,Confederal Group of the European United Left -...,2004-07-20,...,2,3,1,2,1,2,5,2,5,5
3,3,28367,Gabriele,ALBERTINI,"ALBERTINI, Gabriele",yes,Italy,Forza Italia,Group of the European People's Party (Christia...,2004-07-20,...,1,1,1,1,2,1,1,5,5,5
4,4,28512,James Hugh,ALLISTER,"ALLISTER, James Hugh",yes,United Kingdom,Traditional Unionist,Non-attached Members,2004-07-20,...,4,4,4,4,4,4,4,4,4,4


In [ ]:
EP6_subset = EP6_2[['WebisteEpID','Country','EPG',]]

In [ ]:
EP6_subset.head()

,WebisteEpID,Country,EPG
0,28469,Cyprus,Confederal Group of the European United Left -...
1,28302,Poland,Independence/Democracy Group
2,28975,Italy,Confederal Group of the European United Left -...
3,28367,Italy,Group of the European People's Party (Christia...
4,28512,United Kingdom,Non-attached Members


In [ ]:
# Fill NaN values with empty strings
EP6.fillna('', inplace=True)


In [ ]:
import numpy as np


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

from nltk import word_tokenize, WordNetLemmatizer#, pos_tag, pos_tag_sents, sent_tokenize
from nltk.corpus import stopwords#, wordnet

class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
      res=[]
      for token in word_tokenize(doc):
        word=''.join(filter(str.isalnum, token))
        if ((word not in self.ignore_tokens) and (len(self.wnl.lemmatize(word))>2) and not (any(char.isdigit() for char in word))): 
          res.append(self.wnl.lemmatize(word))
      return res

stop_words = stopwords.words('english')
#stop_words.extend(["more","stop","words"])

tokenizer = LemmaTokenizer()
token_stop = list(set(tokenizer(' '.join(stop_words))))
token_stop.sort()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## Preprocessing of the data EP6

In [ ]:
EP6.isnull().sum()


Unnamed: 0          0
Vote ID             0
raw_text            0
type_of_vote        0
main_policy_name    0
                   ..
95280               0
95281               0
95704               0
96101               0
23852               0
Length: 947, dtype: int64

In [ ]:
EP6.shape

(1253, 947)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
# define the columns to transform
text_columns = ['raw_text2', 'type_of_vote', 'main_policy_name', 'committee_code']

# define the transformer
transformer = ColumnTransformer(
    transformers=[
        ('raw_text2', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'raw_text2'),
        ('type_of_vote', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'type_of_vote'),
        ('main_policy_name', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'main_policy_name'),
        ('committee_code', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'committee_code')
    ])

# fit and transform the data
tfidf_array = transformer.fit_transform(EP6[text_columns])




/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# get the feature names for the raw_text column

raw_text_feature_names = transformer.named_transformers_['raw_text2'].get_feature_names_out()
features2 = transformer.named_transformers_['type_of_vote'].get_feature_names_out()
features3 = transformer.named_transformers_['main_policy_name'].get_feature_names_out()
features4 = transformer.named_transformers_['committee_code'].get_feature_names_out()

# print the feature names
print(raw_text_feature_names)
print(features2)
print(features3)
print(features4)

['abandonment' 'ability' 'able' ... 'young' 'young people' 'zone']
['joint' 'joint motion' 'joint motion resolution' 'legislative'
 'legislative resolution' 'motion' 'motion resolution'
 'motion resolution whole' 'report' 'resolution' 'resolution whole'
 'whole']
['affair' 'agriculture' 'budget' 'civil' 'civil liberty'
 'civil liberty justice' 'economics' 'environment' 'environment public'
 'environment public health' 'foreign' 'foreign security'
 'foreign security policy' 'health' 'home' 'home affair' 'juridical'
 'juridical affair' 'justice' 'justice home' 'justice home affair'
 'liberty' 'liberty justice' 'liberty justice home' 'policy' 'public'
 'public health' 'security' 'security policy']
['afet' 'agri' 'budg' 'econ' 'envi' 'juri' 'libe']


In [ ]:
print(tfidf_array.shape)

(1253, 4388)


In [ ]:
from sklearn.decomposition import TruncatedSVD
number_dimensions = 45
lsa_model = TruncatedSVD(n_components=number_dimensions, random_state=420)
lsa_comps = lsa_model.fit_transform(tfidf_array)
print(lsa_model.explained_variance_ratio_)
print(lsa_model.explained_variance_ratio_.sum())

[0.08878754 0.12248197 0.07653322 0.06793096 0.0641246  0.05728278
 0.05354004 0.0495635  0.03293544 0.02861935 0.02655948 0.02349873
 0.02154625 0.02001938 0.01713696 0.01587952 0.0084174  0.00790274
 0.0071001  0.00643547 0.00570467 0.00511605 0.00477595 0.00462822
 0.00420155 0.00401022 0.00392292 0.00370708 0.00356567 0.00331549
 0.00322059 0.00311401 0.00296691 0.00285861 0.00276918 0.00258087
 0.00246918 0.00241332 0.00240079 0.00232857 0.00224494 0.00219623
 0.00208964 0.002024   0.00196762]
0.8768877369731358


In [ ]:
lsa_comps

array([[ 3.35799426e-02,  1.82763305e-01,  1.52046132e-01, ...,
        -1.01518088e-02, -3.53224417e-03, -7.89433617e-03],
       [ 1.99209267e-01,  2.87702322e-01,  1.63999879e-01, ...,
         5.35760559e-03,  5.13025694e-02, -4.32247174e-02],
       [ 6.87359908e-02,  5.65952535e-01,  5.99535453e-02, ...,
        -1.04308055e-02,  9.01133457e-03,  4.40842327e-03],
       ...,
       [ 3.67298388e-01,  8.21879084e-01,  8.65533724e-01, ...,
         5.96606744e-03, -5.23978418e-02,  6.71537655e-03],
       [ 2.19052289e-01,  7.70381334e-01,  1.81302001e-01, ...,
        -1.22963929e-02, -4.15329118e-02,  9.17549813e-03],
       [ 1.16946504e+00,  5.33830244e-01, -6.06125880e-01, ...,
        -1.53634605e-04,  9.67932640e-02, -2.32825917e-02]])

## Create a model for each MEP--> EP6
Changes:
> max_features = 9000 the avg accuracy is 69% with logistic regression and 40 on SVD

> max_features = 9000 the avg accuracy is 71% with logistic regression and 40 on SVD


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Create a dictionary to store MEP names as keys and accuracy values as values
accuracy_results = {}

# Loop through each MEP column and train a separate model for each MEP
for col in EP6.columns[7:]:
    # Step 1: Prepare the data
    X = lsa_comps # Input features (TF-IDF + truncated SVD features)
    y = EP6[col].values # Target variable (voting behavior of the MEP)
    
    # Check the number of unique classes in the target variable
    num_classes = len(np.unique(y))
    if num_classes < 2:
        print(f"Skipping MEP {col} due to only one class present.")
        continue
    
    # Step 2: Split the data into training and testing sets
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Step 3: Choose and configure the model
    model = Pipeline([
        ('scaler', StandardScaler()), # You can apply additional preprocessing steps as needed
        ('classifier', RandomForestClassifier()) # Change the classifier as needed (e.g., DecisionTreeClassifier, RandomForestClassifier, SVC, MLPClassifier)
    ])
    
    # Step 4: Train the model
    accuracy_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    importances = np.zeros(X.shape[1])

    for train_idx, test_idx in cv.split(X, y):
        model.fit(X[train_idx], y[train_idx])
        importances += model.named_steps['classifier'].feature_importances_

    importances /= cv.n_splits
    feature_names = ['feature_' + str(i) for i in range(X.shape[1])]
    feature_importances = dict(zip(feature_names, importances))

    accuracy_results[col] = {'accuracy': np.mean(accuracy_scores), 'feature_importances': feature_importances}
# Convert the accuracy_results dictionary to a DataFrame
accuracy_df = pd.DataFrame.from_dict(accuracy_results, orient='index')

# Add the MEP column to the DataFrame
accuracy_df = accuracy_df.reset_index().rename(columns={'index': 'MEP'})

# Split the feature_importances column into separate columns for each feature
accuracy_df = accuracy_df.join(pd.DataFrame(accuracy_df['feature_importances'].to_dict()).T)


# Drop the original feature_importances column
accuracy_df = accuracy_df.drop(columns=['feature_importances'])
   


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/mod

Skipping MEP 72754 due to only one class present.


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/mod

In [ ]:
accuracy_df.describe()

,accuracy,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44
count,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,...,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000,939.000000
mean,0.732678,0.043619,0.052228,0.027526,0.025111,0.024696,0.021103,0.021581,0.021086,0.026417,...,0.018288,0.018110,0.019872,0.019087,0.019988,0.020149,0.019474,0.018128,0.019421,0.019225
std,0.113998,0.028597,0.021387,0.007945,0.006563,0.005681,0.003755,0.004228,0.003087,0.009268,...,0.004104,0.003277,0.006519,0.005395,0.005010,0.004936,0.005068,0.004394,0.004128,0.004217
min,0.404653,0.014595,0.014078,0.014952,0.011526,0.013139,0.012398,0.011116,0.013962,0.015388,...,0.006766,0.006879,0.006984,0.008578,0.005871,0.008438,0.008161,0.006897,0.010413,0.008528
25%,0.652846,0.024608,0.036938,0.021770,0.020207,0.020325,0.018643,0.018725,0.019016,0.020180,...,0.015680,0.015947,0.016869,0.016249,0.017584,0.016992,0.016106,0.015301,0.016534,0.016605
50%,0.750999,0.031144,0.047245,0.025380,0.024318,0.024135,0.020606,0.021018,0.020693,0.022719,...,0.018501,0.018239,0.019279,0.018882,0.020012,0.019889,0.019173,0.018590,0.019447,0.019339
75%,0.814824,0.052011,0.063415,0.031506,0.029121,0.028272,0.023137,0.023884,0.022793,0.029820,...,0.020830,0.020137,0.021597,0.021030,0.022090,0.022482,0.022814,0.020855,0.022161,0.021672
max,0.993619,0.145170,0.130980,0.061294,0.049036,0.055906,0.043520,0.046162,0.038213,0.065466,...,0.032619,0.035067,0.067852,0.059976,0.050127,0.048376,0.039922,0.040242,0.033716,0.035953


In [20]:
# Get the minimum and maximum accuracy values
min_accuracy = accuracy_df['accuracy'].min()
max_accuracy = accuracy_df['accuracy'].max()
median_accuracy = accuracy_df['accuracy'].median()
mean_accuracy = accuracy_df['accuracy'].mean()

# Print the results
print(f"Minimum Accuracy: {min_accuracy}")
print(f"Maximum Accuracy: {max_accuracy}")
print(f"Median Accuracy: {median_accuracy}")
print(f"Mean Accuracy: {mean_accuracy}")


Minimum Accuracy: 0.4046533864541832
Maximum Accuracy: 0.9936191235059761
Median Accuracy: 0.7509992031872509
Mean Accuracy: 0.7326780868008266
